# import

In [1]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline


# directories


In [2]:
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   

plot_dir = Path('.')/'plot'
plot_dir.mkdir(exist_ok=True,parents=True)

# read exemple

In [3]:
dept = '38'
td007 =pd.read_csv(data_dir/dept/'td007_paroi_opaque.csv',dtype=str)
td007 =pd.read_csv(data_dir/dept/'td007_paroi_opaque.csv',dtype=str)
td006 =pd.read_csv(data_dir/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/dept/'td001_dpe.csv',dtype=str)
td008 =pd.read_csv(data_dir/dept/'td008_baie.csv',dtype=str)


In [4]:
from assets_orm import DPEMetaData

In [5]:
meta = DPEMetaData()
self=meta

In [6]:
from td007_processing import merge_td007_tr_tv,postprocessing_td007
from td008_processing import merge_td008_tr_tv,postprocessing_td008

In [7]:
td008 = merge_td008_tr_tv(td008)
td008 = postprocessing_td008(td008)

In [9]:
table = merge_td007_tr_tv(td007)
table = postprocessing_td007(table,td008)

In [ ]:
td008_sel = td008_vitree.loc[td008_vitree.coefficient_transmission_thermique_baie > 0]
Ubaie_avg = agg_pond_avg(td008_sel, 'coefficient_transmission_thermique_baie', 'max_surface',
                         'td001_dpe_id').to_frame('Ubaie_avg')

In [11]:
# SURFACES

td008_porte = td008.loc[td008.cat_baie_simple_infer=='porte']

td008_vitree = td008.loc[td008.cat_baie_simple_infer.isin(['baie vitrée',
                                                   
                                            'paroi en brique de verre ou polycarbonate'])]
surf_vitree=td008_vitree.groupby('td001_dpe_id')['surfacexnb_baie_calc'].sum()
surf_vitree.name = 'surface_vitree_totale'
surf_porte=td008_porte.groupby('td001_dpe_id')['surfacexnb_baie_calc'].sum()
surf_porte.name = 'surface_porte_totale'

surf_vitree_orient=td008_vitree.pivot_table(index='td001_dpe_id',columns='orientation_infer',values='surfacexnb_baie_calc',aggfunc='sum')

surf_vitree_orient.columns=[f'surfaces_vitree_orientee_{el.lower().replace(" ","_")}' for el in surf_vitree_orient.columns]

td007_murs = td007_m.loc[td007_m.tr014_type_parois_opaque_id.isin(['TR014_002','TR014_001'])]
td007_pb = td007_m.loc[td007_m.tr014_type_parois_opaque_id=='TR014_003']
td007_ph = td007_m.loc[td007_m.tr014_type_parois_opaque_id=='TR014_004']

surf_mur=td007_murs.groupby('td001_dpe_id')[['surface_paroi_opaque_infer',
                                             'surface_paroi_opaque_deperditive_infer','surface_paroi_opaque_exterieur_infer']].sum()
surf_pb=td007_pb.groupby('td001_dpe_id')[['surface_paroi_opaque_infer',
                                             'surface_paroi_opaque_deperditive_infer']].sum()

surf_ph=td007_pb.groupby('td001_dpe_id')[['surface_paroi_opaque_infer',
                                             'surface_paroi_opaque_deperditive_infer']].sum()

surf_mur.columns =['surf_murs_totale','surf_murs_deper','surf_murs_ext']
surf_pb.columns =['surf_pb_totale','surf_pb_deper']
surf_ph.columns =['surf_ph_totale','surf_ph_deper']


quantitatif = pd.concat([surf_mur,surf_pb,surf_ph,surf_vitree,surf_porte,surf_vitree_orient],axis=1)
quantitatif[quantitatif<0]=np.nan

quantitatif['ratio_surface_vitree']=quantitatif.surface_vitree_totale/quantitatif.surf_murs_ext
is_surf_ext = quantitatif.surf_murs_ext==0
ratio_surface_vitree = quantitatif['ratio_surface_vitree']
anomaly_ratio_surface_vitree = ratio_surface_vitree>0.95
quantitatif.loc[anomaly_ratio_surface_vitree,'ratio_surface_vitree']=np.nan

KeyError: 'td001_dpe_id'